#### **Solucionar el problema de Sudoku** ####
#### **Mezcla de Combustibles con Restricciones Ambientales y de Producción** ####
#### **Asignatura:** Programación para Analítica Prescriptiva y de Apoyo a la Decisión ####
#### **Instructor:** Dr. Josué Domínguez Guerrero ####
#### **Alumno:** Aminadab Córdova Acosta ####

In [2]:
# 1. Importación de librerias requeridas
import random # Para generar números aleatorios
from pyomo.environ import * # Para crear el modelo de optimización
from pyomo.opt import SolverFactory, TerminationCondition # Para resolver el modelo

# --------------------------------------------------
# 2. Función para generar pistas aleatorias válidas
# --------------------------------------------------
def generar_pistas_aleatorias(n_pistas):
    pistas = {}
    intentos = 0
    while len(pistas) < n_pistas and intentos < 500:
        i = random.randint(1, 9)
        j = random.randint(1, 9)
        if (i, j) in pistas:
            continue
        k = random.randint(1, 9)

        if all([
            all(k != pistas.get((i, jj), 0) for jj in range(1, 10)),
            all(k != pistas.get((ii, j), 0) for ii in range(1, 10)),
            all(
                k != pistas.get((ii, jj), 0)
                for ii in range(3*((i-1)//3)+1, 3*((i-1)//3)+4)
                for jj in range(3*((j-1)//3)+1, 3*((j-1)//3)+4)
            )
        ]):
            pistas[(i, j)] = k
        intentos += 1
    return pistas

# --------------------------------------------------
# 3. Bucle para generar una instancia resoluble
# --------------------------------------------------
solucion_encontrada = False
intentos = 0
max_intentos = 10  # Evita ciclos infinitos

while not solucion_encontrada and intentos < max_intentos:
    print(f"\nIntento #{intentos + 1}")
    pistas = generar_pistas_aleatorias(25)  # Puedes cambiar la cantidad
    model = ConcreteModel()

    # Índices
    model.I = Set(initialize=range(1, 10))
    model.J = Set(initialize=range(1, 10))
    model.K = Set(initialize=range(1, 10))

    # Variables binarias
    model.x = Var(model.I, model.J, model.K, domain=Binary)

    # Restricciones
    def cell_unique_rule(model, i, j):
        return sum(model.x[i, j, k] for k in model.K) == 1
    model.cell_unique = Constraint(model.I, model.J, rule=cell_unique_rule)

    def row_unique_rule(model, i, k):
        return sum(model.x[i, j, k] for j in model.J) == 1
    model.row_unique = Constraint(model.I, model.K, rule=row_unique_rule)

    def col_unique_rule(model, j, k):
        return sum(model.x[i, j, k] for i in model.I) == 1
    model.col_unique = Constraint(model.J, model.K, rule=col_unique_rule)

    def block_unique_rule(model, br, bc, k):
        return sum(
            model.x[i, j, k]
            for i in range(3*br+1, 3*br+4)
            for j in range(3*bc+1, 3*bc+4)
        ) == 1
    model.block_unique = Constraint(
        range(3), range(3), model.K, rule=block_unique_rule
    )

    # Pistas fijas
    model.clues = ConstraintList()
    for (i, j), val in pistas.items():
        model.clues.add(model.x[i, j, val] == 1)

    # Objetivo: minimizar movimientos (nuevas celdas)
    model.obj = Objective(
        expr=sum(
            model.x[i, j, k]
            for i in model.I for j in model.J for k in model.K
            if (i, j) not in pistas or pistas[(i, j)] != k
        ),
        sense=minimize
    )

    # Resolver
    solver = SolverFactory('glpk')
    results = solver.solve(model, tee=False)

    if results.solver.termination_condition == TerminationCondition.optimal:
        solucion_encontrada = True
        print("¡Sudoku resuelto correctamente!\n")
        print("Sudoku completado:")
        for i in model.I:
            fila = []
            for j in model.J:
                for k in model.K:
                    if value(model.x[i, j, k]) > 0.5:
                        fila.append(str(k))
                        break
            print(" ".join(fila))
    else:
        print("Sudoku sin solución, generando otro...")
        intentos += 1

if not solucion_encontrada:
    print("\nNo se logró generar un Sudoku resoluble tras varios intentos.")


Intento #1
¡Sudoku resuelto correctamente!

Sudoku completado:
7 9 1 6 3 8 2 4 5
8 5 4 9 2 7 1 6 3
2 3 6 1 5 4 9 8 7
5 6 7 2 9 3 4 1 8
4 1 2 5 8 6 3 7 9
3 8 9 4 7 1 5 2 6
6 2 3 7 1 5 8 9 4
9 4 8 3 6 2 7 5 1
1 7 5 8 4 9 6 3 2
